In [3]:
import torch
import torch.nn as nn
from torch.nn import functional as F
import mmap
import random
import pickle
import argparse

parser = argparse.ArgumentParser(description='This is a demonstration program')

# Here we add an argument to the parser, specifying the expected type, a help message, etc.
# parser.add_argument('-batch_size', type=str, required=True, help='Please provide a batch_size')

# args = parser.parse_args()

# Now we can use the argument value in our program.
# print(f'batch size: {args.batch_size}')
device = 'cuda' if torch.cuda.is_available() else 'cpu'

# batch_size = args.batch_size # to use the batch_size cmd arg -> python file_name.py -batch_size 32
batch_size = 32
block_size = 128
max_iters = 100
learning_rate = 3e-4
eval_iters = 100
n_embd = 384
n_head = 4
n_layer = 4
dropout = 0.2

print(device)

cuda


In [6]:
words = ""
with open("/content/drive/MyDrive/Cap_Project/tiny.txt", 'r', encoding='utf-8') as f:
    text = f.read()
    words = text.split()  # Split the text into words using whitespace as the delimiter

# Get unique words
unique_words = sorted(set(words))

vocab_size = len(unique_words)


In [13]:
word_to_int = {word: i for i, word in enumerate(unique_words)}
int_to_word = {i: word for i, word in enumerate(unique_words)}

encode = lambda s: [word_to_int[word] for word in s]
decode = lambda l: ' '.join([int_to_word[i] for i in l])

In [5]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [18]:
import random
import torch
import mmap

# Assuming you've already defined word_to_int and encode functions as before

def get_random_chunk(split, unique_words, block_size, batch_size, device):
    filename = "/content/drive/MyDrive/Cap_Project/tiny.txt" if split == 'train' else "/content/drive/MyDrive/Cap_Project/tiny.txt"
    with open(filename, 'rb') as f:
        with mmap.mmap(f.fileno(), 0, access=mmap.ACCESS_READ) as mm:
            # Determine the file size and a random position to start reading
            file_size = len(mm)
            file_size_limit = max(0, file_size - block_size * batch_size)
            start_pos = random.randint(0, file_size_limit)


            # Seek to the random position and read the block of text
            mm.seek(start_pos)
            block = mm.read(block_size * batch_size - 1)

            # Decode the block to a string, ignoring any invalid byte sequences
            decoded_block = block.decode('utf-8', errors='ignore').replace('\r', '')

            # Tokenize the decoded block into words
            words = decoded_block.split()

            # Convert words to integers using word_to_int
            data = torch.tensor(encode(words), dtype=torch.long)

    return data


def get_batch(split, unique_words, block_size, batch_size, device):
    data = get_random_chunk(split, unique_words, block_size, batch_size, device)
    ix = torch.randint(len(data) - block_size, (batch_size,))
    x = torch.stack([data[i:i + block_size] for i in ix])
    y = torch.stack([data[i + 1:i + block_size + 1] for i in ix])
    x, y = x.to(device), y.to(device)
    return x, y


In [19]:
@torch.no_grad()
def estimate_loss():
    out = {}
    model.eval()

    for split in ['train', 'val']:
        losses = torch.zeros(eval_iters)

        for k in range(eval_iters):
            X, Y = get_batch(split, unique_words, block_size, batch_size, device)
            logits, loss = model(X, Y)
            losses[k] = loss.item()

        out[split] = losses.mean()

    model.train()
    return out


In [20]:
import torch
import torch.nn as nn
import torch.nn.functional as F

class Head(nn.Module):
    def __init__(self, head_size):
        super().__init__()
        self.key = nn.Linear(n_embd, head_size, bias=False)
        self.query = nn.Linear(n_embd, head_size, bias=False)
        self.value = nn.Linear(n_embd, head_size, bias=False)
        self.register_buffer('tril', torch.tril(torch.ones(block_size, block_size)))
        self.dropout = nn.Dropout(dropout)

    def forward(self, x):
        B, T, C = x.shape
        k = self.key(x)
        q = self.query(x)
        wei = q @ k.transpose(-2, -1) * k.shape[-1] ** -0.5
        wei = wei.masked_fill(self.tril[:T, :T] == 0, float('-inf'))
        wei = F.softmax(wei, dim=-1)
        wei = self.dropout(wei)
        v = self.value(x)
        out = wei @ v
        return out

class MultiHeadAttention(nn.Module):
    def __init__(self, num_heads, head_size):
        super().__init__()
        self.heads = nn.ModuleList([Head(head_size) for _ in range(num_heads)])
        self.proj = nn.Linear(head_size * num_heads, n_embd)
        self.dropout = nn.Dropout(dropout)

    def forward(self, x):
        out = torch.cat([h(x) for h in self.heads], dim=-1)
        out = self.dropout(self.proj(out))
        return out

class FeedForward(nn.Module):
    def __init__(self, n_embd):
        super().__init__()
        self.net = nn.Sequential(
            nn.Linear(n_embd, 4 * n_embd),
            nn.ReLU(),
            nn.Linear(4 * n_embd, n_embd),
            nn.Dropout(dropout),
        )

    def forward(self, x):
        return self.net(x)

class Block(nn.Module):
    def __init__(self, n_embd, n_head):
        super().__init__()
        head_size = n_embd // n_head
        self.sa = MultiHeadAttention(n_head, head_size)
        self.ffwd = FeedForward(n_embd)
        self.ln1 = nn.LayerNorm(n_embd)
        self.ln2 = nn.LayerNorm(n_embd)

    def forward(self, x):
        y = self.sa(x)
        x = self.ln1(x + y)
        y = self.ffwd(x)
        x = self.ln2(x + y)
        return x

class GPTLanguageModel(nn.Module):
    def __init__(self, vocab_size):
        super().__init__()
        self.token_embedding_table = nn.Embedding(vocab_size, n_embd)
        self.position_embedding_table = nn.Embedding(block_size, n_embd)
        self.blocks = nn.Sequential(*[Block(n_embd, n_head=n_head) for _ in range(n_layer)])
        self.ln_f = nn.LayerNorm(n_embd)
        self.lm_head = nn.Linear(n_embd, vocab_size)

        self.apply(self._init_weights)

    def _init_weights(self, module):
        if isinstance(module, nn.Linear):
            torch.nn.init.normal_(module.weight, mean=0.0, std=0.02)
            if module.bias is not None:
                torch.nn.init.zeros_(module.bias)
        elif isinstance(module, nn.Embedding):
            torch.nn.init.normal_(module.weight, mean=0.0, std=0.02)

    def forward(self, index, targets=None):
        B, T = index.shape

        tok_emb = self.token_embedding_table(index)
        pos_emb = self.position_embedding_table(torch.arange(T, device=device))
        x = tok_emb + pos_emb
        x = self.blocks(x)
        x = self.ln_f(x)
        logits = self.lm_head(x)

        if targets is None:
            loss = None
        else:
            B, T, C = logits.shape
            logits = logits.view(B * T, C)
            targets = targets.view(B * T)
            loss = F.cross_entropy(logits, targets)

        return logits, loss

    def generate(self, index, max_new_tokens):
        for _ in range(max_new_tokens):
            index_cond = index[:, -block_size:]
            logits, loss = self.forward(index_cond)
            logits = logits[:, -1, :]
            probs = F.softmax(logits, dim=-1)
            index_next = torch.multinomial(probs, num_samples=1)
            index = torch.cat((index, index_next), dim=1)
        return index

model = GPTLanguageModel(vocab_size)
m = model.to(device)


In [21]:
# Create a PyTorch optimizer
optimizer = torch.optim.AdamW(model.parameters(), lr=learning_rate)

for iter in range(max_iters):
    print(iter)
    if iter % eval_iters == 0:
        losses = estimate_loss()
        print(f"step: {iter}, train loss: {losses['train']:.3f}, val loss: {losses['val']:.3f}")

    # Sample a batch of data
    xb, yb = get_batch('train', unique_words, block_size, batch_size, device)

    # Evaluate the loss
    logits, loss = model.forward(xb, yb)
    optimizer.zero_grad(set_to_none=True)
    loss.backward()
    optimizer.step()

print(loss.item())

# Save the model
with open('model-01.pkl', 'wb') as f:
    torch.save(model.state_dict(), f)

print('model saved')


0
step: 0, train loss: 5.561, val loss: 5.561
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
0.09632234275341034
model saved


In [24]:
prompt = 'Academic & Career Entrance Admission Requirements'
prompt_tokens = prompt.split()  # Tokenize the prompt into words
context = torch.tensor(encode(prompt_tokens), dtype=torch.long, device=device)
generated_tokens = decode(m.generate(context.unsqueeze(0), max_new_tokens=100)[0].tolist())
generated_text = ' '.join(generated_tokens)  # Join the generated word tokens into a string
print(generated_text)


A c a d e m i c   &   C a r e e r   E n t r a n c e   A d m i s s i o n   R e q u i r e m e n t s   B i o l o g y   C h e m i s t r y   P h y s i c s   C o m p u t e r   S k i l l s   S e l f - M a n a g e m e n t   S t u d e n t s   m a y   s e l e c t   c o u r s e s   b a s e d   o n   t h e i r   n e e d s .   A . C . E .   i s   r e c o g n i z e d   t h r o u g h o u t   t h e   O n t a r i o   C o l l e g e   s y s t e m .   A d m i s s i o n   R e q u i r e m e n t s   i n   C h a t h a m .   C o u r s e s   r a n g e   f r o m :   T e c h n i c a l   M a t h e m a t i c s   C o m m u n i c a t i o n s   ( E n g l i s h )   B i o l o g y   C h e m i s t r y   P h y s i c s   C o m p u t e r   S k i l l s   t h i s   p r o g r a m ,   c a l l   5 1 9 - 3 5 4 - 9 1 0 0   e x t .   3 5 8 6   i n   C h a t h a m .   C o u r s e s   r a n g e   f r o m :   T e c h n i c a l   M a t h e m a t i c s   C o m m u n i c a t i o n s   ( E n g l i s h )   B i o l o g y   C h e m i s t r y 